In [57]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [64]:
def extractHTML(HTMLcards, username, tweets_list, max_tweets):
    if '@' not in username:
        username = '@' + username
        
    for card in HTMLcards:
        if len(tweets_list) == max_tweets:
            break
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        tweet = card.find_element(By.XPATH, './/div[2]/div[2]/div[2]').text
        if handle != username:
            continue
        if tweet in tweets_list:
            continue
        tweets_list.append(tweet)

    return tweets_list

In [82]:
def getTweets(username, max_tweets=50):
    # extract username from link, if link given instead of username
    if 'twitter.com/' in username:
        username = username.split("/")[-1]

    # configure Chrome options for headless browsing
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # run Chrome in headless mode
    chrome_options.add_argument("--disable-gpu")  # disable GPU acceleration

    # create a WebDriver instance with headless Chrome
    driver = webdriver.Chrome(options=chrome_options)
    
    # navigate to Twitter page and wait to load
    driver.get(f"https://twitter.com/{username}")
    time.sleep(2)

    # get tweets

    tweets_list = []
    retry_counter = 0
    while len(tweets_list) < max_tweets:
        # break after retrying three times if error
        if retry_counter >= 3:
            break
        
        num_tweets = len(tweets_list)
        # print (num_tweets)

        window_height = driver.execute_script("return window.innerHeight;")

        # extract tweets from HTML cards
        HTMLcards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
        tweets = extractHTML(HTMLcards, username, tweets_list=tweets_list, max_tweets=max_tweets)
        tweets_list = tweets

        # check for loading error and if all tweets extracted
        if num_tweets == len(tweets_list):
            retry_counter += 1
            # print (f'retry: {retry_counter}')
        else:
            retry_counter = 0
            
        # scroll down and wait to load
        driver.execute_script(f"window.scrollBy(0, {5 * window_height});")
        time.sleep(1.25)

    driver.quit()

    return tweets_list

In [83]:
tweets = getTweets('https://twitter.com/shalin_1', max_tweets=50)

In [84]:
print (len(tweets))
print (tweets[40])

50
i've been reluctant to say it all season, but my title charging Gunners are actually gonna complete this title charge
